### TempView vs. GlobalTempView
PySpark SQL temporary views are session-scoped and will not be available if the session that creates it terminates. If you want to have a temporary view that is shared among all sessions and keep alive until the Spark application terminates, you can create a global temporary view using createGlobalTempView()

In [1]:
import findspark
findspark.init()

In [2]:
import pyspark
from pyspark.sql import SparkSession

In [13]:
spark = SparkSession.builder.master("local[*]").appName("ViewsApp").getOrCreate()
print(spark)
print(spark.sparkContext)

<SparkContext master=local[*] appName=ViewsApp>


In [4]:
data = [('James', 'Smith', '1998-04-01', 3000),
        ('Michael', 'Rose', '2000-05-19', 4000),
        ('Maria', 'Jones', '1999-12-01', 4000)]
columns = ["first", "last", "birth_date", "salary"]
df = spark.createDataFrame(data=data, schema=columns)

In [5]:
df.show()

+-------+-----+----------+------+
|  first| last|birth_date|salary|
+-------+-----+----------+------+
|  James|Smith|1998-04-01|  3000|
|Michael| Rose|2000-05-19|  4000|
|  Maria|Jones|1999-12-01|  4000|
+-------+-----+----------+------+



In [6]:
spark.catalog.dropTempView("tempView")
df.createOrReplaceTempView("tempView")
spark.catalog.listTables()

[Table(name='tempView', catalog=None, namespace=[], description=None, tableType='TEMPORARY', isTemporary=True)]

In [7]:
spark.sql("SELECT * FROM tempView").show()

+-------+-----+----------+------+
|  first| last|birth_date|salary|
+-------+-----+----------+------+
|  James|Smith|1998-04-01|  3000|
|Michael| Rose|2000-05-19|  4000|
|  Maria|Jones|1999-12-01|  4000|
+-------+-----+----------+------+



In [8]:
spark.catalog.dropGlobalTempView("globalTempView")
df.createGlobalTempView("globalTempView")

In [9]:
spark.sql("SELECT * FROM global_temp.globalTempView").show()

+-------+-----+----------+------+
|  first| last|birth_date|salary|
+-------+-----+----------+------+
|  James|Smith|1998-04-01|  3000|
|Michael| Rose|2000-05-19|  4000|
|  Maria|Jones|1999-12-01|  4000|
+-------+-----+----------+------+



## Accessing Views from Another Spark Session
* TempView is not accessible from another session 
* GlobalTempView is accessibe from multiple sessions

In [12]:
sparkNew = spark.newSession()
print(sparkNew)
print(sparkNew.sparkContext)

<SparkContext master=local[*] appName=ViewsApp>


In [14]:
sparkNew.sql("SELECT * FROM global_temp.globalTempView").show()

+-------+-----+----------+------+
|  first| last|birth_date|salary|
+-------+-----+----------+------+
|  James|Smith|1998-04-01|  3000|
|Michael| Rose|2000-05-19|  4000|
|  Maria|Jones|1999-12-01|  4000|
+-------+-----+----------+------+



In [16]:
try:
    sparkNew.sql("SELECT * FROM tempView").show()
except Exception as e:
    print("Temporary View not found!")

Temporary View not found!


## Closing Spark Context

In [17]:
spark.stop()

## New Spark Session (with New Context)
None of the view is accessible because:
* TempView       => Session Based
* GlobalTempView => Context Based

In [18]:
sparkA = SparkSession.builder.master("local[*]").appName("ViewsApp").getOrCreate()
print(sparkA)
print(sparkA.sparkContext)

<SparkContext master=local[*] appName=ViewsApp>


In [20]:
try:
    sparkA.sql("SELECT * FROM tempView").show()
except Exception as e:
    print("Temporary View not found!")

Temporary View not found!


In [21]:
try:
    sparkA.sql("SELECT * FROM global_temp.globalTempView").show()
except Exception as e:
    print("Global Temporary View not found!")

Global Temporary View not found!
